<a href="https://colab.research.google.com/github/zaku2590/signate/blob/main/%E3%82%B2%E3%83%BC%E3%83%A0%E3%81%AE%E5%8B%9D%E6%95%97%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)
sample_submit = pd.read_csv("sample_submit.csv", index_col=0, header=None)

先頭5行表示

In [35]:
train.head()

,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience,blueWins
gameId,,,,,,,,,
0,0,5,8,6,0,0,14536,17256,0
1,1,10,1,5,0,0,14536,17863,0
2,0,3,10,2,0,0,17409,17256,0
3,1,7,10,8,0,0,19558,18201,0
4,0,4,9,4,0,0,17409,17256,0


In [34]:
test.head()

,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience
gameId,,,,,,,,
9,0,7,6,6,0,0,16961,18201
15,0,6,6,6,2,1,18513,18021
18,1,6,4,3,0,0,13475,17256
23,0,5,4,7,0,0,17409,17256
31,0,10,8,9,0,0,18117,18472


In [19]:
print(train.shape)
print(test.shape)

(8000, 9)
(2000, 8)


欠損の確認

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8000 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   blueFirstBlood       8000 non-null   int64
 1   blueKills            8000 non-null   int64
 2   blueDeaths           8000 non-null   int64
 3   blueAssists          8000 non-null   int64
 4   blueEliteMonsters    8000 non-null   int64
 5   blueDragons          8000 non-null   int64
 6   blueTotalGold        8000 non-null   int64
 7   blueTotalExperience  8000 non-null   int64
 8   blueWins             8000 non-null   int64
dtypes: int64(9)
memory usage: 625.0 KB


In [22]:
train['blueWins'].value_counts()

,count
blueWins,
0,4066
1,3934


統計値

In [23]:
train.describe()

,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience,blueWins
count,8000.000000,8000.000000,8000.00000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,0.513000,6.029750,5.92350,5.881875,0.538625,0.358125,17154.067750,17983.514000,0.491750
std,0.499862,2.623876,2.64892,3.189897,0.629728,0.479479,1820.796171,772.348402,0.499963
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,13475.000000,16650.000000,0.000000
25%,0.000000,4.000000,4.00000,3.000000,0.000000,0.000000,16544.000000,17256.000000,0.000000
50%,1.000000,6.000000,6.00000,6.000000,0.000000,0.000000,17409.000000,18021.000000,0.000000
75%,1.000000,8.000000,8.00000,8.000000,1.000000,1.000000,18274.000000,18472.000000,1.000000
max,1.000000,14.000000,14.00000,17.000000,2.000000,1.000000,20619.000000,20101.000000,1.000000


ブルーの勝利との相関係数

In [25]:
train.corrwith(train["blueWins"])

,0
blueFirstBlood,0.226536
blueKills,0.337541
blueDeaths,-0.237224
blueAssists,0.311647
blueEliteMonsters,0.272810
blueDragons,0.251957
blueTotalGold,0.040489
blueTotalExperience,0.328167
blueWins,1.000000


In [38]:
# 必要なライブラリをインポート
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# 目的変数（y）と説明変数（X）の分離
y = train["blueWins"]  # 青チームが勝った場合1、それ以外は0
X = train.drop(["blueWins"], axis=1)  # blueWins以外の特徴量

# パイプライン作成（標準化 + ロジスティック回帰）
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 特徴量を標準化
    ('logreg', LogisticRegression(solver='lbfgs', max_iter=500))  # ロジスティック回帰モデル
])

# モデル学習
pipeline.fit(X, y)

# テストデータで予測（青チームが勝つ確率）
pred_proba = pipeline.predict_proba(test)[:, 1]  # ポジティブクラスの確率（青チームが勝つ確率）

# しきい値0.5を基準に0または1に変換（必要に応じて）
pred_binary = (pred_proba >= 0.5).astype(int)

# 提出用データの作成
# sample_submit["blueWins"] = pred_proba  # 確率で提出する場合
sample_submit["blueWins"] = pred_binary  # 0または1で提出する場合

# CSVに保存
sample_submit.to_csv("submission.csv", index=False)  # 提出用ファイル保存

# 結果の確認
print("予測確率 (最初の5件):", pred_proba[:5])  # 確率の確認
print("予測結果 (最初の5件):", pred_binary[:5])  # 0または1の確認

予測確率 (最初の5件): [0.45384815 0.55497992 0.41398293 0.32997628 0.57210262]
予測結果 (最初の5件): [0 1 0 0 1]
